In [2]:
###Package import
import pandas as pd
import matplotlib
import datetime

In [7]:
###Data read in
sales = pd.read_csv('sales.csv')
calls = pd.read_csv('calls.csv')
agents = pd.read_csv('agents.csv')
customers = pd.read_csv('customers.csv')
markets = pd.read_csv('markets.csv')

###Null checks
sales.isnull().sum()
calls.isnull().sum()
agents.isnull().sum()
customers.isnull().sum()
markets.isnull().sum()
###No nulls only NaN's from voicemalls

market_id                        0
census_name                      0
common_name                      0
population                       0
days_on_market_median            0
price_new_list_median            0
price_closed_median              0
listings_on_market_count         0
listings_price_increase_count    0
listings_price_decrease_count    0
listings_closed_count            0
dtype: int64

In [14]:
###Best Markets
###A marketing team manager wants to know: What are currently the five best markets for Smiley Real Estate? What evidence do you have for that? Also, did you notice anything on intrest that we could shre with an outside audience (a pure market trend)

###Price Change Rate
markets['price_change_rate'] = markets['listings_price_increase_count']/markets['listings_price_decrease_count'].replace(0, pd.NA)
markets['price_change_rate'] = markets['price_change_rate'].fillna(1)
markets['price_change_rate_rank'] = markets['price_change_rate'].rank(method='dense', ascending=False)
market_price_sort = markets.sort_values(by='price_change_rate_rank', ascending=False)

###Closing Rate
markets['listing_closing_rate'] = markets['listings_closed_count']/markets['listings_on_market_count']
markets['listing_closing_rate_rank'] = markets['listing_closing_rate'].rank(method='dense', ascending=False)
market_close_sort = markets.sort_values(by='listing_closing_rate_rank', ascending=False)

###Price Change Per Day
markets['price_change_per_day'] = (markets['price_closed_median'] - markets['price_new_list_median'])/markets['days_on_market_median']
markets['price_change_per_day_rank'] = markets['price_change_per_day'].rank(method='dense', ascending=False)
markets_price_day_sort = markets.sort_values(by='price_change_per_day_rank', ascending= False)

markets['price_closed_median_rank'] = markets['price_closed_median'].rank(method='dense', ascending=False)
markets['population_rank'] = markets['population'].rank(method='dense', ascending=False)

markets['rate_ranks_combine'] = markets['price_change_rate_rank'] + markets['listing_closing_rate_rank'] + markets['price_change_per_day_rank']
markets['total_ranks_combine'] = markets['price_change_rate_rank'] + markets['listing_closing_rate_rank'] + markets['price_change_per_day_rank'] + markets['population_rank'] + markets['price_closed_median_rank']

markets.sort_values(by='total_ranks_combine', ascending=True, inplace=True)

markets.to_csv('top_markets.csv', index=False)

In [13]:
### Best Agents
#### No multiple sales
num_customer_calls = calls.groupby('customer_id')['call_id'].nunique().reset_index(name='times_called') 

call_merge = pd.merge(
    customers[['customer_id', 'assigned_agent']],
    num_customer_calls,
    how="left",
    on='customer_id'
)

call_sales_merge = pd.merge(
    call_merge,
    sales[['customer_id', 'amount']],
    on= 'customer_id',
    how="left"
)

###Stats Prep
call_sales_merge['amount'] = call_sales_merge['amount'].fillna(0)
sale_customers = call_sales_merge[call_sales_merge['amount'] > 0]
agent_medians = sale_customers.groupby('assigned_agent')[['amount','times_called']].median().reset_index()
agent_medians.rename(columns={'assigned_agent':'agent_id', 'amount':'median_amount_closed', 'times_called':'median_customer_calls'}, inplace=True)

###Agent Merge
agent_merge = pd.merge(
    agents,
    agent_medians,
    how="left",
    on="agent_id"
)

###Asha Fill
agent_merge['customer_reviews_2024'] = agent_merge['customer_reviews_2024'].fillna(agent_merge['customer_reviews_2024'].median())
agent_merge['avg_rating_2024'] = agent_merge['avg_rating_2024'].fillna(agent_merge['avg_rating_2024'].median())
agent_prep = agent_merge

###Weight AVG
C = agent_prep['avg_rating_2024'].mean()
m = 2 ###Min number of reviews
agent_prep['weight_review_avg'] = (agent_prep['avg_rating_2024'] * agent_prep['customer_reviews_2024'] + C*m) / (agent_prep['customer_reviews_2024'] + m)

###Other Math
agent_prep['total_customer_closure_rate'] = (agent_prep['buyer_closes_2024'] + agent_prep['seller_closes_2024']) / (agent_prep['buyer_customers_sent_2024'] + agent_prep['seller_customers_sent_2024'])
agent_prep['total_closes'] = agent_prep['buyer_closes_2024'] + agent_prep['seller_closes_2024']

###Ranking
agent_prep['median_amount_closed_rank'] = agent_prep['median_amount_closed'].rank(method='dense', ascending=False)
agent_prep['median_customer_calls_rank'] = agent_prep['median_customer_calls'].rank(method='dense', ascending=False)
agent_prep['weight_review_avg_rank'] = agent_prep['weight_review_avg'].rank(method='dense', ascending=False)
agent_prep['total_customer_closure_rate_rank'] = agent_prep['total_customer_closure_rate'].rank(method='dense', ascending=False)
agent_prep['total_closes_rank'] = agent_prep['total_closes'].rank(method="dense",ascending=False)
agent_prep['total_ranks'] = agent_prep['median_amount_closed_rank'] + agent_prep['median_customer_calls_rank'] + agent_prep['weight_review_avg_rank'] + agent_prep['total_customer_closure_rate_rank'] + agent_prep['total_closes_rank']
agent_prep['company_rank'] = agent_prep['total_ranks'].rank(method='dense', ascending=True)
agent_prep['agent_ranking'] = agent_prep.groupby('market_id')['total_ranks'].rank(method='dense', ascending=True)

top_agent = agent_prep[agent_prep['agent_ranking'] == 1]
top_agent.to_csv('top_agent.csv', index=False)